In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle

from tqdm import tqdm

out_dir = "../../output/count"
os.makedirs(out_dir, exist_ok=True)


In [2]:
airport = "kbos"
base_dir = '../../datasets/amelia'
traj_version = 'a10v08'



traj_dir = os.path.join(base_dir, f'traj_data_{traj_version}', 'raw_trajectories', f'{airport}')

traj_files = [os.path.join(traj_dir, f) for f in os.listdir(traj_dir)]
print(traj_files[0])

data = pd.read_csv(traj_files[0])
data.head()

../../datasets/amelia/traj_data_a10v08/raw_trajectories/kbos/KBOS_570_1674626400.csv


,Frame,ID,Altitude,Speed,Heading,Lat,Lon,Range,Bearing,Type,Interp,x,y
0,0,1818967,12.5,0.0,171.867920,42.35289,-71.01582,1.444150,-2.797495,1.0,[ORG],-1.359494,-0.487181
1,0,1818783,12.5,0.0,159.954346,42.37469,-71.00399,1.168532,0.430202,1.0,[ORG],1.062057,0.487342
2,0,1819383,12.5,0.0,209.052246,42.36078,-71.01558,0.672141,-2.372738,2.0,[ORG],-0.483073,-0.467348
3,0,1818760,12.5,0.0,183.022723,42.35498,-71.01719,1.277072,-2.652468,2.0,[ORG],-1.127328,-0.600037
4,1,1818967,12.5,0.0,171.877808,42.35289,-71.01582,1.444150,-2.797495,1.0,[ORG],-1.359494,-0.487181


In [3]:
agent_counts = {
    'total': [], '0': [], '1': [], '2': []
}
total_count = 0
for f in tqdm(traj_files):
    data = pd.read_csv(f)

    unique_frames = np.unique(data.Frame)
    for frame in unique_frames:
        agents = data[data.Frame == frame]

        agent_counts['total'].append(agents.shape[0])
        agent_counts['0'].append(np.where(agents.Type == 0.0)[0].shape[0])
        agent_counts['1'].append(np.where(agents.Type == 1.0)[0].shape[0])
        agent_counts['2'].append(np.where(agents.Type == 2.0)[0].shape[0])


100%|██████████| 100/100 [01:27<00:00,  1.14it/s]


In [4]:
agent_count_stats = {}
for k, v in agent_counts.items():
    v = np.asarray(v)
    agent_count_stats[k] = {
        "min": round(v.min().astype(float), 5),
        "max": round(v.max().astype(float), 5),
        "mean": round(v.mean().astype(float), 5),
        "std": round(v.std().astype(float), 5)
    }

    agent_counts[k] = v

agent_count_stats

{'total': {'min': 1.0, 'max': 34.0, 'mean': 8.96946, 'std': 4.90629},
 '0': {'min': 0.0, 'max': 22.0, 'mean': 5.81227, 'std': 4.22062},
 '1': {'min': 0.0, 'max': 12.0, 'mean': 1.09617, 'std': 1.55719},
 '2': {'min': 0.0, 'max': 11.0, 'mean': 2.06102, 'std': 1.55569}}

In [5]:
with open(f'{out_dir}/{airport}_stats.json', 'w') as f:
    json.dump(agent_count_stats, f, indent=2)

with open(f'{out_dir}/{airport}.pkl', 'wb') as handle:
    pickle.dump(agent_counts, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
name = {'total': 'Total', '0': 'Aircraft', '1': 'Vehicle', '2': 'Unknown'}
colors = {'ksea': 'blue', 'kewr': 'red', 'kbos': 'green', 'kmdw': 'orange'}
for key, value in name.items():
    arr = agent_counts[key]

    plt.hist(arr, bins = int(arr.max()), color=colors[airport])
    plt.title(f"{value}")
    plt.xlabel("Number of Agents per Timestep")
    plt.ylabel("Count")
    plt.savefig(f"{out_dir}/{airport}_{key}.png", dpi=600)
    plt.close()

In [7]:
for key, value in name.items():
    arr = agent_counts[key]
    q_lower = np.quantile(arr, 0.05)
    q_upper = np.quantile(arr, 0.95)

    arr = arr[(arr >= q_lower) & (arr <= q_upper)]

    plt.hist(arr, bins = int(arr.max()), color=colors[airport])
    plt.title(f"{value}")
    plt.xlabel("Number of Agents per Timestep")
    plt.ylabel("Count")
    plt.savefig(f"{out_dir}/{airport}_{key}_iqr.png", dpi=600)
    plt.close()